In [1]:
!nvidia-smi

Wed Sep  3 07:07:02 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

!cp /content/gdrive/MyDrive/path/to/data.zip /content

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/gdrive/MyDrive/path/to/data.zip': No such file or directory


In [ ]:
!unzip -q /content/data.zip -d /content/custom_data

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python train_val_split.py --datapath="/content/custom_data" --train_pct=0.9


--2025-08-25 04:01:52--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.13K  --.-KB/s    in 0s      

2025-08-25 04:01:52 (51.9 MB/s) - ‘/content/train_val_split.py’ saved [3203/3203]

Created folder at /content/data/train/images.
Created folder at /content/data/train/labels.
Created folder at /content/data/validation/images.
Created folder at /content/data/validation/labels.
Number of image files: 851
Number of annotation files: 851
Images moving to train: 765
Images moving to validation: 86


In [3]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 66.1 MB/s eta 0:00:00


In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

    # Read class.txt to get class names
    if not os.path.exists(path_to_classes_txt):
        print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
        return
    with open(path_to_classes_txt, 'r') as f:
        classes = []
        for line in f.readlines():
            if len(line.strip()) == 0: continue
            classes.append(line.strip())
    number_of_classes = len(classes)

    # Create data dictionary
    data = {
        'path': '/content/data',
        'train': 'train/images',
        'val': 'validation/images',
        'nc': number_of_classes,
        'names': classes
    }

    # Write data to YAML file
    with open(path_to_data_yaml, 'w') as f:
        yaml.dump(data, f, sort_keys=False)
    print(f'Created config file at {path_to_data_yaml}')

    return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml


Created config file at /content/data.yaml

File contents:

path: /content/data
train: train/images
val: validation/images
nc: 2
names:
- fire
- smoke


In [4]:
!yolo detect train data=/content/data.yaml model=yolov8s.pt epochs=150 imgsz=768

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.191 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=768, int8=False, iou=0.7, keras=False, kobj=1.0, line_wid

In [ ]:
!yolo detect predict model=runs/detect/train3/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.185 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs

image 1/86 /content/data/validation/images/000ef5db-Datacluster_Fire_and_Smoke_Sample_30.jpg: 640x480 1 fire, 79.7ms
image 2/86 /content/data/validation/images/00e194d4-fire.71.png: 416x640 1 fire, 78.7ms
image 3/86 /content/data/validation/images/0432429d-fire.567.png: 448x640 1 fire, 1 smoke, 78.7ms
image 4/86 /content/data/validation/images/0516bd0b-fire.504.png: 448x640 1 fire, 12.1ms
image 5/86 /content/data/validation/images/0624a350-fire.500.png: 448x640 1 fire, 18.9ms
image 6/86 /content/data/validation/images/0910a1da-fire.717.png: 448x640 2 fires, 12.4ms
image 7/86 /content/data/validation/images/0a333cd5-fire.201.png: 448x640 1 fire, 1 smoke, 15.8ms
image 8/86 /content/data/validation/images/161882ea-fire.505.png: 416x640 (no detections), 17.8ms
image 9/86 /content/data/validation/images/1758b04d-fire.680.png:

In [ ]:
!zip -r predict.zip /content/runs/detect/predict

  adding: content/runs/detect/predict/ (stored 0%)
  adding: content/runs/detect/predict/150540-798555688_small.avi (deflated 6%)
  adding: content/runs/detect/predict/photo-1523861751938-121b5323b48b.jpg (deflated 5%)
  adding: content/runs/detect/predict/234515_small.avi (deflated 1%)
  adding: content/runs/detect/predict/CampFire-1059476842.jpg (deflated 0%)
  adding: content/runs/detect/predict/download.jpg (deflated 3%)


In [ ]:
from google.colab import files
files.download("predict.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from ultralytics import YOLO

# Load your best model
model = YOLO("best.pt")

# Run inference on an image
results = model.predict(source="HomeFires.jpg", show=True, save=True, conf=0.25)



image 1/1 /content/HomeFires.jpg: 480x640 5 fires, 46.1ms
Speed: 1.4ms preprocess, 46.1ms inference, 2.1ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs/detect/predict4


In [ ]:
from ultralytics import YOLO

model = YOLO("best.pt")

# Run inference on a video
results = model.predict(source="test_video.mp4", show=True, save=True, conf=0.25)



WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/307) /content/test_video.mp4: 384x640 1 fire, 43.5ms
video 1/1 (frame 2/307) /content/test_video.mp4: 384x640 1 fire, 10.7ms
video 1/1 (frame 3/307) /content/test_video.mp4: 384x640 1 fire, 10.7ms
video 1/1 (frame 4/307) /content/test_video.mp4: 384x640 1 fire, 10.7ms
video 1/1 (frame 5/307) /content/test_video.mp4: 384x640 1 fire, 10.7ms
video 1/1 (frame 6/307) /content/test_video.mp4: 384x640 1 fire, 10.9ms
video 1/1 (frame 7/307) 

In [ ]:
!unzip -q /content/data.zip -d /content/fire_test_folder

In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO("best.pt")

# Run inference on all images & videos in the folder
results = model.predict(
    source="/content/fire_test_folder",  # folder with .jpg, .png, .mp4 etc.
    show=True,     # show in interactive window (works locally, in Colab it won't popup)
    save=True,     # saves output with detections
    conf=0.4       # confidence threshold
)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

image 1/6 /content/fire_test_folder/CampFire-1059476842.webp: 640x640 2 fires, 927.4ms
image 2/6 /content/fire_test_folder/download.jpg: 384x640 1 fire, 439.4ms
image 3/6 /content/fire_test_folder/photo-1523861751938-121b5323b48b.jpg: 448x640 1 fire, 453.0ms
video 4/6 (frame 1/573) /content/fire_test_folder/150540-798555688_small.mp4: 384x640 1 fire, 368.9ms
video 4/6 (frame 2/57

KeyboardInterrupt: 